In [1]:
import xsimlab as xs

In [2]:
xs.process
class Hello:
    
    a = xs.variable()

In [3]:
Hello.a

_CountingAttr(counter=20, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, on_setattr=None, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.IN: 'in'>, 'global_name': None, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})

In [29]:
import numpy as np

In [30]:
import matplotlib.pyplot as plt

In [31]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

In [32]:
from phydra.components.variables import SV

from phydra.components.fluxes.basic import LinearDecay

In [58]:
@phydra.comp
class DoubleForcing:
    fx1 = phydra.forcing(setup_func="forcing1")
    
    fx2 = phydra.forcing(setup_func="forcing2")
    
    def forcing1(self):
        @np.vectorize
        def fx(time):
            return 1
        return fx
    
    def forcing2(self):
        @np.vectorize
        def fx2(time):
            return fx(time)
        return fx2

In [59]:
m = phydra.create({"FX":DoubleForcing, "A":SV})

m

<xsimlab.Model (5 processes, 6 inputs)>
Core
    solver_type     [in]
Time
    time            [in] ('input_time',) sequence of time points fo...
A
    var_label       [in] label / basic state variable
    var_init        [in] initial value / basic state variable
FX
    fx1_label       [in] label / 
    fx2_label       [in] label / 
Solver

In [60]:
a = phydra.setup(solver="odeint", time=np.arange(1,10), model=m,
            input_vars={"FX":{"fx1_label":"f1", f"fx2_label":"f2"},
                       "A":{"var_label":"A","var_init":1}})

In [61]:
a.xsimlab.run(model=m)

initializing model backend
Initializing Model Time
Initializing component A
Initializing component FX
finalizing: cleanup


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [62]:
@phydra.comp
class GroupVariable:
    var = phydra.variable(flux='testflux', description='hello')
    
    @phydra.flux(group='hello')
    def testflux(self, var):
        # print(type(self.testflux_label))
        return 0.1

In [5]:
TestModel = phydra.create({'comp':GroupVariable})

In [6]:
TestModel

<xsimlab.Model (4 processes, 4 inputs)>
Core
    solver_type     [in]
Time
    time            [in] ('input_time',) sequence of time points fo...
comp
    var_label       [in] label / hello
    var_init        [in] initial value / hello
Solver

In [7]:
test_setup = phydra.setup(solver='odeint', model=TestModel,
                          time= np.arange(0,10),
                          input_vars={
                             'comp':{'var_label':'X', 'var_init':0.1},
                         })

In [8]:
test_setup

<xarray.Dataset>
Dimensions:            (clock: 2, input_time: 10)
Coordinates:
  * clock              (clock) int64 0 1
Dimensions without coordinates: input_time
Data variables:
    comp__var_label    <U1 'X'
    comp__var_init     float64 0.1
    Core__solver_type  <U6 'odeint'
    Time__time         (input_time) int64 0 1 2 3 4 5 6 7 8 9
Attributes:
    __xsimlab_output_vars__:  comp__var_value,comp__testflux_value,comp__test...

In [9]:
outarray = test_setup.xsimlab.run(model=TestModel)

<xarray.Variable ()>
array('odeint', dtype='<U6') None <class 'NoneType'>
() odeint {} {}
<xarray.Variable ()>
array('X', dtype='<U1')
Attributes:
    description:       label / hello
    Phydra_store_out:  True None <class 'NoneType'>
() X {'description': 'label / hello', 'Phydra_store_out': True} {}
<xarray.Variable ()>
array(0.1)
Attributes:
    description:       initial value / hello
    Phydra_store_out:  True
    _FillValue:        nan None <class 'NoneType'>
() 0.1 {'description': 'initial value / hello', 'Phydra_store_out': True, '_FillValue': nan} {}
<xarray.Variable (clock: 2)>
array([0, 1])
Attributes:
    __xsimlab_output_clock__:  1
    __xsimlab_master_clock__:  1 None <class 'NoneType'>
('clock',) [0 1] {'__xsimlab_output_clock__': 1, '__xsimlab_master_clock__': 1} {}
<xarray.Variable (input_time: 10)>
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Attributes:
    description:  sequence of time points for which to solve the model None <class 'NoneType'>
('input_time',) [0 1 2 3 

In [10]:
outarray#.to_netcdf('test.nc')

<xarray.Dataset>
Dimensions:               (clock: 2, input_time: 10, time: 10)
Coordinates:
  * clock                 (clock) int64 0 1
Dimensions without coordinates: input_time, time
Data variables:
    Core__solver_type     <U6 'odeint'
    Time__time            (input_time) int64 0 1 2 3 4 5 6 7 8 9
    comp__testflux_label  <U13 'comp_testflux'
    comp__testflux_value  (time) float64 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
    comp__var_init        float64 0.1
    comp__var_label       <U1 'X'
    comp__var_value       (time) float64 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0

In [16]:
outarray.to_netcdf('test.nc')

<xarray.Variable ()>
array('odeint', dtype='<U6') None <class 'NoneType'>
() odeint {} {'chunks': (), 'compressor': None, 'filters': None}
<xarray.Variable (input_time: 10)>
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Attributes:
    description:  sequence of time points for which to solve the model None <class 'NoneType'>
('input_time',) [0 1 2 3 4 5 6 7 8 9] {'description': 'sequence of time points for which to solve the model'} {'chunks': (10,), 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None}
<xarray.Variable (clock: 2)>
array([0, 1])
Attributes:
    __xsimlab_master_clock__:  1
    __xsimlab_output_clock__:  1 None <class 'NoneType'>
('clock',) [0 1] {'__xsimlab_master_clock__': 1, '__xsimlab_output_clock__': 1} {'chunks': (2,), 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None}
<xarray.Variable ()>
array('comp_testflux', dtype='<U13')
Attributes:
    Phydra_store_out:  True
    description:       label re

NotImplementedError: variable None has a _FillValue specified, but _FillValue is not yet supported on unicode strings: https://github.com/pydata/xarray/issues/1647